<a href="https://colab.research.google.com/github/Genometric/MSPC/blob/dev/Benchmark/PlotBenchmarkings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme()
sns.set_context("paper")

In [16]:
df = pd.read_csv("benchmarking_results.tsv", sep="\t")
df_replicates = pd.read_csv("benchmarking_results_replicates.tsv", sep="\t")

In [3]:
df

,mspc_version,experiment_id,replicate_count,interval_count,runtime_seconds,peak_physical_memory_usage_bytes,peak_paged_memory_usage_bytes,peak_virtual_memory_usage_bytes
0,v1.1,AFF1_ENCSR426URK,2,24476,694.212512,58585088,49950720,181129216
1,v1.1,ATF3_ENCSR000BNU,2,18573,564.243153,46534656,38195200,180473856
2,v1.1,BACH1_ENCSR000EGD,2,10834,548.407660,36323328,27287552,166383616
3,v1.1,CCAR2_ENCSR598GER,2,1558,566.185291,26505216,17879040,165072896
4,v1.1,CDC5L_ENCSR121PFY,2,5187,725.557262,28839936,20471808,166383616
...,...,...,...,...,...,...,...,...
95,v5.5.0,ZC3H11A_ENCSR000EFR,2,12168,1.435666,73846784,34955264,2204070535168
96,v5.5.0,ZNF316_ENCSR167KBO,2,141755,8.508641,187899904,154914816,2204072960000
97,v5.5.0,ZNF318_ENCSR334HSW,2,1769,1.069475,65294336,27209728,2204071378944
98,v5.5.0,ZNF592_ENCSR249BHQ,2,56525,4.503288,113672192,78872576,2204072828928


In [ ]:
versions_groups = df.groupby("mspc_version")
len(versions_groups)

In [5]:
releases = {}
for release in versions_groups.groups:
  g = versions_groups.get_group(release)
  g_ = g.copy()
  g_ = g_.drop('mspc_version', axis=1)
  releases[release] = g_

In [6]:
def get_increase_percentage(cur_val, pre_val):
  return ((pre_val - cur_val) / abs(pre_val)) * 100

def get_improvement_foldchange(cur_val, pre_val):
  return pre_val/cur_val

In [11]:
changes = []
odf = releases["v5.5.0"]
for idx, row in releases["v1.1"].iterrows():
  other = odf.loc[(odf["interval_count"] == row["interval_count"]) & (odf["replicate_count"] == row["replicate_count"])]
  if len(other) == 1:
    changes.append({
        "experiment_id": row["experiment_id"],
        "replicate_count": row["replicate_count"],
        "interval_count": row["interval_count"],
        "runtime_change_percent": get_improvement_foldchange(other["runtime_seconds"].to_list()[0], row["runtime_seconds"]),
        "peak_physical_memory_usage_change_percent": get_improvement_foldchange(other["peak_physical_memory_usage_bytes"].to_list()[0], row["peak_physical_memory_usage_bytes"]),
        "peak_paged_memory_usage_change_percent": get_improvement_foldchange(other["peak_paged_memory_usage_bytes"].to_list()[0], row["peak_paged_memory_usage_bytes"]),
        "peak_virtual_memory_usage_change_percent": get_improvement_foldchange(other["peak_virtual_memory_usage_bytes"].to_list()[0], row["peak_virtual_memory_usage_bytes"])
    })
  elif len(other) == 0:
    print("Could not find a match; interval count " + str(row["interval_count"]) + ", and replicate count "+ str(row["replicate_count"]))
    continue
  else:
    print("More than one match found; interval count " + str(row["interval_count"]) + ", and replicate count " + str(row["replicate_count"]))
    continue

In [8]:
len(changes)

50

In [9]:
changes_df = pd.DataFrame(changes)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
fig.set_dpi(600)


changes_df.sort_values("runtime_change_percent", ascending=False, inplace=True)
sns.barplot(data=changes_df, x="interval_count", y="runtime_change_percent", order=changes_df["interval_count"], facecolor=(0.2, 0.2, 0.4, 1), ax=ax)
ax.set(xlabel="Interval count", ylabel="Runtime Improvement fold change\n")
ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',') + "x"))
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.title.set_text("Runtime improvement between\nMSPC v1 and v6")

plt.tight_layout()
plt.savefig('benchmarks.png')

Convert peak physical memory usage scale from bytes to mega bytes.

In [17]:
peak_mem_col_name = "peak_physical_memory_usage_bytes"
df_replicates[peak_mem_col_name] = df_replicates[peak_mem_col_name].div(1024*1024)

In [29]:
def hide_even_num_ticks(axis):
  [l.set_visible(False) for (i,l) in enumerate(axis.xaxis.get_ticklabels()) if i % 2 != 0]

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,4))
fig.set_dpi(600)

sns.boxplot(x=df_replicates["replicate_count"], y=df_replicates["runtime_seconds"], ax=ax[0])
ax[0].set(xlabel="\n\nReplicate count", ylabel="Runtime (seconds)")
ax[0].title.set_text("Changes in runtime\nw.r.t increase in the replicate count")

sns.boxplot(x=df_replicates["replicate_count"], y=df_replicates[peak_mem_col_name], ax=ax[1])
ax[1].set(xlabel="\n\nReplicate count", ylabel="Peak Physical Memory Usage\n(Megabyte)")
ax[1].title.set_text("\nChanges in memory requirement\nw.r.t increase in the replicate count")

hide_even_num_ticks(ax[0])
hide_even_num_ticks(ax[1])

# Set color of boxes in the boxplot.
# for i,box in enumerate(ax[1].artists):
#     box.set_edgecolor('black')
#     box.set_facecolor((0.0, 0.0, 0.0, 0))
#     # iterate over whiskers and median lines
#     for j in range(6 * i, 6 * (i + 1)):
#          ax[1].lines[j].set_color('black')

plt.tight_layout()
plt.savefig('scalability.png')